# Tarea Modulo 5: Algoritmos de aprendizaje supervizado - modelos de regresión lineal

## Profesor: Jose Pino Jauregui
### Alumno: Cristóbal Saldías Rojas

<hr>

Se creó un modelo de regesión lineal múltiple para predecir los años de escolaridad en base a las variables: región (region), zona (zona), sexo (sexo), edad (edad), analfabetismo (e1), nivel educacional más alto (e6a), si ha trabajado alguna vez (o4), ocupación u oficio (oficio1), a qué negocio se dedica (rama1), salario líquido del trabajo principal (y1), si ha recibido bonificaciones en los últimos 12 meses (y4a), la edad que tenía cuando nació el primer hijo (s5), nacionalidad (r1a), ingreso del trabajo (ytrabajocor), escolaridad (esc) y condición de actividad económica (activ).

Para esto, la tarea se dividió en 3 archivos con el fin de que quedase ordenado todo con una estructura lógica en cuanto a qué se realiza. Los archivos son:
- __Tarea_P1__: en este archivo se carga la encuesta CASEN - 2017 completa y se seleccionan las columunas con las que se desea trabajar. Una vez seleccionadas las columnas, éstas se exportan en un archivo excel llamado "CASEN2017_seleccionada.xlsx".
- __Tarea_P2__: este archivo contiene los procesos de imputación y eliminación de outliers, en otras palabras, el análisis exploratorio de datos. Además se exponen diferentes gráficos para observar la distribución de la información en cada una de las variables seleccionadas previamente.
- __Tarea_P3__: el tercer archivo, se implementa el modelo de regresión lineal. Se va ajustando el conjunto de datos, con el fin de optimizar el modelo. Posterior a ésto, se revisan los supuestos asociados al modelo de regresión lineal.
- __Tarea_P4__: se implementa el modelo de regresión lineal múltiple mediante la librería _scikit-lear_ en base al deataset generado en el tercer archivo. 
<br>

Cabe mencionar que, para que opere todo de forma correcta, los archivos deben estar dentro de la misma carpeta y se deben ejecutar en el orden de: Tarea_P1, Tarea_P2 y Tarea_P3. Además se cuenta con imágenes anexas que permiten describir ciertos sucesos que se dieron, éstas se encuentran dentro de la carpeta _Imagenes_.

<hr>

# Parte 1: Selección de la información
En este primer archivo, lo que se realizó fue montar toda la encuesta CASEN - 2017 almacenada en el archivo _Casen 2017.sav_. Posterior esto, se creó y exportó un dataset con las columnas y descripción de éstas, para seleccionar aquellas variables de interés que fuesen necesarias para la implementación del modelo de regresión lineal múltiple. Dicho archivo exportado, se emplea para el análisis exploratorio de datos del segundo archivo de la tarea.

<hr>

In [1]:
# Librerias necesarias
import pyreadstat # importar archivos en formato .SAV (SPSS)
import pandas as pd # Manipulación de datos

In [2]:
# Lectura de la encuesta CASEN - 2017
casen, meta = pyreadstat.read_sav("Casen 2017.sav", apply_value_formats = True)
print("CASEN cargada!")

CASEN cargada!


In [3]:
# pd.set_option("display.max_columns", None) visualizacion de todas las columnas
casen.head()

,folio,o,id_vivienda,hogar,region,provincia,comuna,zona,expr,expc,...,hh_d_estado,hh_d_servbas,hh_d_entorno,hh_d_accesi,hh_d_medio,hh_d_appart,hh_d_tsocial,hh_d_seg,pobreza_multi_4d,pobreza_multi_5d
0,1.101100e+11,1.0,1.101100e+09,1.0,Región de Tarapacá,Iquique,Iquique,Urbano,39.0,33.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN
1,1.101100e+11,1.0,1.101100e+09,1.0,Región de Tarapacá,Iquique,Iquique,Urbano,39.0,33.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pobre,Pobre
2,1.101100e+11,1.0,1.101100e+09,1.0,Región de Tarapacá,Iquique,Iquique,Urbano,39.0,33.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN
3,1.101100e+11,2.0,1.101100e+09,1.0,Región de Tarapacá,Iquique,Iquique,Urbano,39.0,33.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN
4,1.101100e+11,1.0,1.101100e+09,1.0,Región de Tarapacá,Iquique,Iquique,Urbano,39.0,33.0,...,1.0,1.0,1.0,0.0,1.0,NaN,1.0,0.0,No pobre,NaN


In [4]:
casen.shape

(216439, 804)

## Se crea un DF con el nombre de las columnas, etiqueta (pregunta realizada) y el tipo de dato que posee

In [5]:
etiquetas = pd.DataFrame(list((meta.column_names_to_labels).items()), columns=['Columna', 'Etiqueta'])

tipos= pd.DataFrame(casen.dtypes, columns= ['Tipos'])
tipos['Columna'] = tipos.index
tipos= tipos.reset_index(drop= True)

dicc_datos= pd.merge(etiquetas, tipos, on= 'Columna')
dicc_datos

,Columna,Etiqueta,Tipos
0,folio,Identificación hogar (comuna area seg viv hogar),float64
1,o,Orden,float64
2,id_vivienda,Identificador de la vivienda (comuna area seg ...,float64
3,hogar,Identificación del hogar en la vivienda,float64
4,region,Región,category
...,...,...,...
799,hh_d_appart,Hogar carente en apoyo y participación social,float64
800,hh_d_tsocial,Hogar carente en trato igualitario,float64
801,hh_d_seg,Hogar carente en seguridad,float64
802,pobreza_multi_4d,Situación de pobreza multidimensional (4 dimen...,category


## Se exporta el DF de las etiquetas

In [6]:
df_copia = dicc_datos
# Se crea el objeto que escribe el dataset
writer = pd.ExcelWriter('Etiquetas_CASEN2017.xlsx')
# Se setea el data set sin los indices que atribuye pandas
df_copia.to_excel(writer, index=False)
# Se guarda el dataset en cuestion
writer.save()
print('Se ha exportado el excel.')

Se ha exportado el excel.


## Se seleccionan las columnas con las que se desea trabajar
- región (region)
- zona (zona)
- sexo (sexo)
- edad (edad)
- analfabetismo (e1)
- nivel educacional más alto (e6a)
- si ha trabajado alguna vez (o4)
- ocupación u oficio (oficio1)
- a qué negocio se dedica (rama1)
- salario líquido del trabajo principal (y1)
- si ha recibido bonificaciones en los últimos 12 meses (y4a)
- la edad que tenía cuando nació el primer hijo (s5)
- nacionalidad (r1a)
- ingreso del trabajo (ytrabajocor)
- escolaridad (esc)
- condición de actividad económica (activ)

In [7]:
data = casen[["region","zona","sexo","edad","e1","e6a","o4","oficio1","rama1","y1","y4a","s5","r1a","ytrabajocor","esc","activ"]]
print(data.shape)
data.head()

(216439, 16)


,region,zona,sexo,edad,e1,e6a,o4,oficio1,rama1,y1,y4a,s5,r1a,ytrabajocor,esc,activ
0,Región de Tarapacá,Urbano,Mujer,56.0,"Sí, lee y escribe",Educación Media Científico-Humanista,NaN,Trabajadores no calificado,Comercio al por mayor y al por menor,NaN,NaN,NaN,Otra nacionalidad. Especifique país,250000.0,12.0,Ocupados
1,Región de Tarapacá,Urbano,Mujer,21.0,"Sí, lee y escribe",Educación Básica,NaN,Trabajadores no calificado,Hogares privados con servicio doméstico,200000.0,0.0,NaN,Otra nacionalidad. Especifique país,200000.0,7.0,Ocupados
2,Región de Tarapacá,Urbano,Hombre,24.0,"Sí, lee y escribe",Educación Media Científico-Humanista,NaN,Trabajadores de los servicios y vendedores de ...,Hoteles y restaurantes,270000.0,15000.0,NaN,Chilena (Exclusiva),281750.0,12.0,Ocupados
3,Región de Tarapacá,Urbano,Hombre,28.0,"Sí, lee y escribe",Educación Media Científico-Humanista,NaN,Trabajadores de los servicios y vendedores de ...,Hoteles y restaurantes,300000.0,15000.0,NaN,Otra nacionalidad. Especifique país,311750.0,12.0,Ocupados
4,Región de Tarapacá,Urbano,Hombre,26.0,"Sí, lee y escribe",Técnico Nivel Superior Completo (Carreras 1 a ...,NaN,Trabajadores no calificado,Hogares privados con servicio doméstico,320000.0,20000.0,25.0,Otra nacionalidad. Especifique país,341667.0,15.0,Ocupados


## Se exportan los registros seleccionados

In [8]:
df_copia = data
# Se crea el objeto que escribe el dataset
writer = pd.ExcelWriter('CASEN2017_seleccionada.xlsx')
# Se setea el data set sin los indices que atribuye pandas
df_copia.to_excel(writer, index=False)
# Se guarda el dataset en cuestion
writer.save()
print('Se ha exportado el excel.')

Se ha exportado el excel.
